# In this notebook , I will value interest rate swap using QuantLib library

In [1]:
from QuantLib import *

In [3]:
rf = 0.01
libor_rate = 0.02
day_count = Actual365Fixed()
reference_date = Date(20,10,2015)
Settings.instance().evaluationDate = reference_date

In [4]:
curve = YieldTermStructureHandle(FlatForward(reference_date, rf, day_count))

In [5]:
libor_curve = YieldTermStructureHandle(FlatForward(reference_date, libor_rate, day_count))

In [17]:
libor3M_index = USDLibor(Period(3, Months), libor_curve)

#### Now we need the fixed leg and floating leg for the swap. We construct the fixed rate and floating rate leg schedules as below.

##### Several things need attention:

- calendar.advance: append the days according the calendar chosen
- ModifiedFollowing: choose the first business day after the given holiday unless it belongs to a different month, in which case choose the first business day before the holiday.
- DateGeneration.Forward: Forward from effective date to termination date

In [14]:
calendar = UnitedStates()

# move forward 5 days from the reference date using US business calendar (holidays and off-days in the Calendar considered)
settle_date = calendar.advance(reference_date, 5, Days)

# move 10 years from the settlement date.
maturity_date = calendar.advance(settle_date, 10, Years)

# period of payment every 6 months
fixed_leg_tenor = Period(6, Months)
fixed_schedule = Schedule(settle_date, maturity_date,
                         fixed_leg_tenor, calendar, 
                         ModifiedFollowing, ModifiedFollowing,
                         DateGeneration.Forward, False)

# period of payment every 3 months
float_leg_tenor = Period(3, Months)
float_schedule = Schedule(settle_date, maturity_date, 
                         float_leg_tenor, calendar, 
                         ModifiedFollowing, ModifiedFollowing,
                         DateGeneration.Forward, False)

In [19]:
notional = 1000000
fixed_rate = 0.025
fixed_leg_daycount = Actual360()
float_spread = 0.004
float_leg_daycount = Actual360()

ir_swap = VanillaSwap(VanillaSwap.Payer, notional, fixed_schedule, 
                     fixed_rate, fixed_leg_daycount, float_schedule, 
                     libor3M_index, float_spread, float_leg_daycount)

In [22]:
swap_engine = DiscountingSwapEngine(curve)
ir_swap.setPricingEngine(swap_engine)

In [31]:
for i, cf in enumerate(ir_swap.leg(0)):
    print("%2d %-18s %10.2f"%(i+1, cf.date(), cf.amount()))

 1 April 27th, 2016     12708.33
 2 October 27th, 2016   12708.33
 3 April 27th, 2017     12638.89
 4 October 27th, 2017   12708.33
 5 April 27th, 2018     12638.89
 6 October 29th, 2018   12847.22
 7 April 29th, 2019     12638.89
 8 October 28th, 2019   12638.89
 9 April 27th, 2020     12638.89
10 October 27th, 2020   12708.33
11 April 27th, 2021     12638.89
12 October 27th, 2021   12708.33
13 April 27th, 2022     12638.89
14 October 27th, 2022   12708.33
15 April 27th, 2023     12638.89
16 October 27th, 2023   12708.33
17 April 29th, 2024     12847.22
18 October 28th, 2024   12638.89
19 April 28th, 2025     12638.89
20 October 27th, 2025   12638.89


In [32]:
for i, cf in enumerate(ir_swap.leg(1)):
    print("%2d %-18s %10.2f"%(i+1, cf.date(), cf.amount()))

 1 January 27th, 2016    6076.05
 2 April 27th, 2016      6009.86
 3 July 27th, 2016       6009.86
 4 October 27th, 2016    6076.05
 5 January 27th, 2017    6076.05
 6 April 27th, 2017      5943.69
 7 July 27th, 2017       6009.86
 8 October 27th, 2017    6076.05
 9 January 29th, 2018    6208.42
10 April 27th, 2018      5811.34
11 July 27th, 2018       6009.86
12 October 29th, 2018    6208.42
13 January 28th, 2019    6009.86
14 April 29th, 2019      6009.86
15 July 29th, 2019       6009.86
16 October 28th, 2019    6009.86
17 January 27th, 2020    6009.86
18 April 27th, 2020      6009.86
19 July 27th, 2020       6009.86
20 October 27th, 2020    6076.05
21 January 27th, 2021    6076.05
22 April 27th, 2021      5943.69
23 July 27th, 2021       6009.86
24 October 27th, 2021    6076.05
25 January 27th, 2022    6076.05
26 April 27th, 2022      5943.69
27 July 27th, 2022       6009.86
28 October 27th, 2022    6076.05
29 January 27th, 2023    6076.05
30 April 27th, 2023      5943.69
31 July 27